In [1]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd

current_directory = os.listdir()
match = [match for match in current_directory if 'for_modeling' in match]
match = match[0]


df = pd.read_csv(match, index_col=0)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99178 entries, 0 to 99177
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               99178 non-null  object 
 1   LittleFallsGage    99174 non-null  float64
 2   01646500-00060     99174 non-null  float64
 3   SenecaGage         99178 non-null  float64
 4   01645000-00060     99178 non-null  float64
 5   PointofRocksGage   99178 non-null  float64
 6   01638500-00060     99178 non-null  int64  
 7   EdwardsGage        99178 non-null  float64
 8   ShhepardstownGage  99177 non-null  float64
 9   01618000-00060     99177 non-null  float64
 10  HancockGage        99178 non-null  float64
 11  01613000-00060     99178 non-null  int64  
 12  SpringfieldGage    99178 non-null  float64
 13  01608500-00060     99178 non-null  int64  
dtypes: float64(10), int64(3), object(1)
memory usage: 11.4+ MB


In [3]:
# lag variables
# train, test split

df['PointofRocksGage'] = df['PointofRocksGage'].shift(16)
df['EdwardsGage'] = df['EdwardsGage'].shift(12)
df['ShhepardstownGage'] = df['ShhepardstownGage'].shift(32)
df['HancockGage'] = df['HancockGage'].shift(60)
df['SpringfieldGage'] = df['SpringfieldGage'].shift(68)

df.drop(df.columns[[2,4,6,9,11,13]], axis=1, inplace=True)

In [50]:
df = df.dropna()

In [51]:
df_train = df.sample(frac=0.8,random_state=200)
df_test = df.drop(df_train.index)

In [52]:
Y_train = df_train['LittleFallsGage']
X_train = df_train.drop(columns=['Date', 'LittleFallsGage'])

In [53]:
Y_test = df_test['LittleFallsGage']
X_test = df_test.drop(columns=['Date', 'LittleFallsGage'])

In [ ]:
# need to break them out by seperate hourly predictions
#X_train_17hrs = X_train['SpringfieldGage']

In [54]:
model_17hrs = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation = 'relu', kernel_regularizer='l1', input_shape = (1,)),  # input shape required
    tf.keras.layers.Dense(3, activation = 'relu', kernel_regularizer='l1'),
    tf.keras.layers.Dense(1, activation = 'relu', kernel_regularizer='l1')
])

model_17hrs.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error')

In [55]:
# Train
model_17hrs.fit(X_train['SpringfieldGage'], Y_train, epochs=50, verbose=1)

Epoch 1/50
2478/2478 [==============================] - 2s 687us/step - loss: 2.9203 - mean_squared_error: 2.8894
Epoch 2/50
2478/2478 [==============================] - 2s 684us/step - loss: 0.0786 - mean_squared_error: 0.0428
Epoch 3/50
2478/2478 [==============================] - 2s 684us/step - loss: 0.0781 - mean_squared_error: 0.0429
Epoch 4/50
2478/2478 [==============================] - 2s 677us/step - loss: 0.0776 - mean_squared_error: 0.0430
Epoch 5/50
2478/2478 [==============================] - 2s 685us/step - loss: 0.0769 - mean_squared_error: 0.0429
Epoch 6/50
2478/2478 [==============================] - 2s 685us/step - loss: 0.0762 - mean_squared_error: 0.0429
Epoch 7/50
2478/2478 [==============================] - 2s 742us/step - loss: 0.0753 - mean_squared_error: 0.0430
Epoch 8/50
2478/2478 [==============================] - 2s 710us/step - loss: 0.0738 - mean_squared_error: 0.0430
Epoch 9/50
2478/2478 [==============================] - 2s 700us/step - loss: 0.0710 - m

In [56]:
pred = model_17hrs.predict(X_test['SpringfieldGage'])

In [76]:
compare_df = pd.DataFrame(pred, columns=['Prediction'])
compare_df['TrueValue'] = Y_test.values
# used values bcuz...
# the index from the Y_test index is used to map it onto the new compare_df
# but the compare_df got its index form the prediction so it starts at 0 unlike Y_test
# so it only matches up rows correctly when the indexes match, which is few

In [80]:
compare_df.head(500)

,Prediction,TrueValue
0,4.718582,4.99
1,4.718582,4.98
2,4.698079,4.95
3,4.684410,4.93
4,4.670741,4.92
5,4.663907,4.91
6,4.622900,4.86
7,4.595563,4.83
8,4.588728,4.82
9,4.581894,4.82


In [ ]:
# not good enough
# need to add more features, like past 3 obs. 

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [70]:
pd.set_option('display.max_rows', 500)